In [ ]:
import pandas as pd
import numpy as np

In [ ]:
democracy = pd.read_csv("../democracy-index-eiu.csv")
unemp = pd.read_csv("../unemployment analysis.csv")
education = pd.read_csv("../Country-Income-and-Education-Level.csv")

In [ ]:
democracy = democracy[democracy["Year"] == 2022][["Entity", "Democracy score"]]
democracy.head(5)

,Entity,Democracy score
14,Afghanistan,0.320
29,Africa,4.075
45,Albania,6.410
61,Algeria,3.660
77,Angola,3.960


In [ ]:
import pandas as pd

# Assuming your data is in a DataFrame called democracy
rename_dict = {
    "Cape Verde": "Cabo Verde",
    "Congo": "Congo, Rep.",
    "Democratic Republic of Congo": "Congo, Dem. Rep.",
    "Czechia": "Czech Republic",
    "East Timor": "Timor-Leste",
    "Egypt": "Egypt, Arab Rep.",
    "Gambia": "Gambia, The",
    "Hong Kong": "Hong Kong SAR, China",
    "Iran": "Iran, Islamic Rep.",
    "Kyrgyzstan": "Kyrgyz Republic",
    "Laos": "Lao PDR",
    "North Korea": "Korea, Dem. People's Rep.",
    "South Korea": "Korea, Rep.",
    "Russia": "Russian Federation",
    "Slovakia": "Slovak Republic",
    "Syria": "Syrian Arab Republic",
    "Turkey": "Turkiye",
    "Venezuela": "Venezuela, RB",
    "Yemen": "Yemen, Rep.",
    "West Bank and Gaza": "Palestine"
}

# Rename the countries in the 'democracy' DataFrame
democracy = democracy.replace({"Entity": rename_dict})


In [ ]:
unemp = unemp[["Country Name", "2021"]]
unemp.head(5)

,Country Name,2021
0,Africa Eastern and Southern,8.11
1,Afghanistan,13.28
2,Africa Western and Central,6.84
3,Angola,8.53
4,Albania,11.82


In [ ]:
unemp["Country Name"] = unemp["Country Name"].replace({"West Bank and Gaza": "Palestine"})

In [ ]:
education = education[["Country", "Education Index"]]

rename_dict = {
    "Bolivia (Plurin. State of)": "Bolivia",
    "Congo": "Congo, Rep.",
    "Côte d’Ivoire": "Cote d'Ivoire",
    "Czechia": "Czech Republic",
    "Dem. Rep. of the Congo": "Congo, Dem. Rep.",
    "Egypt": "Egypt, Arab Rep.",
    "Gambia": "Gambia, The",
    "China Hong Kong SAR": "Hong Kong SAR, China",
    "Iran (Islamic Republic of)": "Iran, Islamic Rep.",
    "Kyrgyzstan": "Kyrgyz Republic",
    "Lao People's Dem. Rep": "Lao PDR",
    "Republic of Moldova": "Moldova",
    "Macedonia": "North Macedonia",
    "Slovakia": "Slovak Republic",
    "Republic of Korea": "Korea, Rep.",
    "United Rep. of Tanzania": "Tanzania",
    "Turkey": "Turkiye",
    "United States of America": "United States",
    "Venezuela (Boliv. Rep. of)": "Venezuela, RB",
    "Viet Nam": "Vietnam",
    "Yemen": "Yemen, Rep.",
    "Bahamas": "Bahamas, The",
    "Taiwan Province of China": "Taiwan",
    "Lao People\\'s Dem. Rep": "Lao PDR",
    "Dem. People\\'s Rep. Korea": "Korea, Dem. People's Rep.",
}

education['Country'] = education['Country'].replace(rename_dict)

In [ ]:
full = democracy.merge(unemp, how="outer", left_on = "Entity", right_on = "Country Name")
full[full.isna().sum(axis=1) > 0]

,Entity,Democracy score,Country Name,2021
1,Africa,4.075000,NaN,NaN
7,Asia,4.228298,NaN,NaN
54,Europe,7.398000,NaN,NaN
120,Oceania,7.460000,NaN,NaN
143,South America,6.250833,NaN,NaN
...,...,...,...,...
236,NaN,NaN,Upper middle income,6.76
237,NaN,NaN,St. Vincent and the Grenadines,21.62
238,NaN,NaN,Virgin Islands (U.S.),13.27
239,NaN,NaN,Vanuatu,2.18


In [ ]:
full_full = full.merge(education, how="outer", left_on = "Country Name", right_on = "Country")
full_full[full_full.isna().sum(axis=1) > .0]

,Entity,Democracy score,Country Name,2021,Country,Education Index
1,Africa,4.075000,NaN,NaN,NaN,NaN
2,Asia,4.228298,NaN,NaN,NaN,NaN
3,Europe,7.398000,NaN,NaN,NaN,NaN
4,Oceania,7.460000,NaN,NaN,NaN,NaN
5,South America,6.250833,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
239,NaN,NaN,Vanuatu,2.18,Vanuatu,0.561
240,NaN,NaN,Samoa,9.84,Samoa,0.692
241,NaN,NaN,NaN,NaN,Taiwan,NaN
242,NaN,NaN,NaN,NaN,The former Yugoslav,NaN


In [ ]:
full_full_combined = (
    full_full
    .groupby(full_full[['Entity', 'Country', "Country Name"]].bfill(axis=1).iloc[:, 0])  # Group by a common identifier
    .apply(lambda x: x.ffill().bfill())  # Forward and backward fill within each group
    .drop_duplicates()  # Remove any duplicate rows created in the process
    .reset_index(drop=True)
)

In [ ]:
full_full_combined

,Entity,Democracy score,Country Name,2021,Country,Education Index
0,Afghanistan,0.320000,Afghanistan,13.28,Afghanistan,0.414
1,Africa,4.075000,NaN,NaN,NaN,NaN
2,NaN,NaN,Africa Eastern and Southern,8.11,NaN,NaN
3,NaN,NaN,Africa Western and Central,6.84,NaN,NaN
4,Albania,6.410000,Albania,11.82,Albania,0.745
...,...,...,...,...,...,...
238,NaN,NaN,Virgin Islands (U.S.),13.27,NaN,NaN
239,World,5.293174,World,6.18,NaN,NaN
240,"Yemen, Rep.",1.950000,"Yemen, Rep.",13.57,"Yemen, Rep.",0.360
241,Zambia,5.800000,Zambia,13.03,Zambia,0.557


In [ ]:
def rename_countries(df):
    # Define replacements for country names
    country_replacements = {
        "Bahamas, The": "Bahamas",
        "Congo, Dem. Rep.": "Democratic Republic of the Congo",
        "Congo, Rep.": "Republic of the Congo",
        "Egypt, Arab Rep.": "Egypt",
        "Gambia, The": "Gambia",
        "Hong Kong SAR, China": "Hong Kong",
        "Iran, Islamic Rep.": "Iran",
        "Korea, Dem. People's Rep.": "North Korea",
        "Korea, Rep.": "South Korea",
        "Kyrgyz Republic": "Kyrgyzstan",
        "Lao PDR": "Laos",
        "Macao SAR, China": "Macao",
        "Micronesia, Fed. Sts.": "Micronesia",
        "Russian Federation": "Russia",
        "Slovak Republic": "Slovakia",
        "Syrian Arab Republic": "Syria",
        "Tanzania": "Tanzania",
        "United States": "United States",
        "Venezuela, RB": "Venezuela",
        "Yemen, Rep.": "Yemen"
    }
    df = df.applymap(lambda x: country_replacements.get(x, x) if isinstance(x, str) else x)
    return df

In [ ]:
clean_df = rename_countries(full_full_combined)

In [ ]:
clean_df[["Entity","Country Name", "Country"]] = clean_df[["Entity","Country Name", "Country"]].ffill(axis=1).bfill(axis=1)

In [ ]:
clean_df = clean_df.drop(columns=["Entity", "Country Name"])

In [ ]:
excluded_regions = [
    "Africa", "Africa Eastern and Southern", "Africa Western and Central",
    "Arab World", "Asia", "Caribbean small states", "Central Europe and the Baltics",
    "Early-demographic dividend", "East Asia & Pacific", "East Asia & Pacific (IDA & IBRD countries)",
    "East Asia & Pacific (excluding high income)", "Euro area", "Europe", "Europe & Central Asia",
    "Europe & Central Asia (IDA & IBRD countries)", "Europe & Central Asia (excluding high income)",
    "European Union", "Fragile and conflict affected situations", "Heavily indebted poor countries (HIPC)",
    "High income", "IBRD only", "IDA & IBRD total", "IDA blend", "IDA only", "IDA total",
    "Late-demographic dividend", "Latin America & Caribbean", "Latin America & Caribbean (excluding high income)",
    "Latin America & the Caribbean (IDA & IBRD countries)", "Least developed countries: UN classification",
    "Low & middle income", "Low income", "Lower middle income", "Middle East & North Africa",
    "Middle East & North Africa (IDA & IBRD countries)", "Middle East & North Africa (excluding high income)",
    "Middle income", "North America", "OECD members", "Oceania", "Other small states",
    "Pacific island small states", "Post-demographic dividend", "Pre-demographic dividend",
    "Small states", "South America", "South Asia", "South Asia (IDA & IBRD)",
    "Sub-Saharan Africa", "Sub-Saharan Africa (IDA & IBRD countries)", "Sub-Saharan Africa (excluding high income)",
    "Upper middle income", "World", "The former Yugoslav"
]

In [ ]:
final_df = clean_df[~clean_df["Country"].isin(excluded_regions)]

In [ ]:
final_df = final_df.rename(columns={"2021": "Unemployment Rate", "Democracy score": "Democracy Score"})[["Country", "Democracy Score", "Unemployment Rate", "Education Index"]]

In [ ]:
final_df = final_df.reset_index(drop=True).reset_index()

In [ ]:
final_df = final_df.rename(columns={"index": "Country ID"})

In [ ]:
final_df.to_csv("../country_table_cleaned.csv")

In [ ]:
final_df

,Country ID,Country,Democracy Score,Unemployment Rate,Education Index
0,0,Afghanistan,0.32,13.28,0.414
1,1,Albania,6.41,11.82,0.745
2,2,Algeria,3.66,12.70,0.664
3,3,Angola,3.96,8.53,0.500
4,4,Argentina,6.85,10.90,0.816
...,...,...,...,...,...
184,184,Vietnam,2.73,2.17,0.630
185,185,Virgin Islands (U.S.),NaN,13.27,NaN
186,186,Yemen,1.95,13.57,0.360
187,187,Zambia,5.80,13.03,0.557


In [ ]:
big_countries = pd.read_csv("../countries_data (2).csv")

In [ ]:
big_countries["Country"] = big_countries["Country"].replace({"The Bahamas": "Bahamas", "The Gambia": "Gambia", "Republic of Ireland": "Ireland", "Palestinian National Authority": "Palestine", "S�����������": "Sao Tome and Principe"})
final_df["Country"] = final_df["Country"].replace({"Brunei Darussalam": "Brunei", "Cabo Verde": "Cape Verde", "Cote d'Ivoire": "Ivory Coast", "Timor-Leste": "East Timor", "Turkiye": "Turkey", "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines", "St. Lucia": "Saint Lucia"})

In [ ]:
big_countries

,Country,Agricultural Land( %),Land Area(Km2),Birth Rate,Co2-Emissions,CPI,CPI Change (%),Fertility Rate,Forested Area (%),Gasoline Price,...,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude
0,Afghanistan,0.581,652230.0,32.49,8672.0,149.90,0.023,4.47,0.021,0.70,...,0.784,0.28,38041754.0,0.489,0.093,0.714,0.1112,9797273.0,33.939110,67.709953
1,Albania,0.431,28748.0,11.78,4536.0,119.05,0.014,1.62,0.281,1.36,...,0.569,1.20,2854191.0,0.557,0.186,0.366,0.1233,1747593.0,41.153332,20.168331
2,Algeria,0.174,2381741.0,24.28,150006.0,151.36,0.020,3.02,0.008,0.28,...,0.281,1.72,43053054.0,0.412,0.372,0.661,0.1170,31510100.0,28.033886,1.659626
3,Andorra,0.400,468.0,7.20,469.0,NaN,NaN,1.27,0.340,1.51,...,0.364,3.33,77142.0,NaN,NaN,NaN,NaN,67873.0,42.506285,1.521801
4,Angola,0.475,1246700.0,40.73,34693.0,261.73,0.171,5.52,0.463,0.97,...,0.334,0.21,31825295.0,0.775,0.092,0.491,0.0689,21061025.0,-11.202692,17.873887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Venezuela,0.245,912050.0,17.88,164175.0,2740.27,2.549,2.27,0.527,0.00,...,0.458,1.92,28515829.0,0.597,NaN,0.733,0.0880,25162368.0,6.423750,-66.589730
191,Vietnam,0.393,331210.0,16.75,192668.0,163.52,0.028,2.05,0.481,0.80,...,0.435,0.82,96462106.0,0.774,0.191,0.376,0.0201,35332140.0,14.058324,108.277199
192,Yemen,0.446,527968.0,30.45,10609.0,157.58,0.081,3.79,0.010,0.92,...,0.810,0.31,29161922.0,0.380,NaN,0.266,0.1291,10869523.0,15.552727,48.516388
193,Zambia,0.321,752618.0,36.19,5141.0,212.31,0.092,4.63,0.652,1.40,...,0.275,1.19,17861030.0,0.746,0.162,0.156,0.1143,7871713.0,-13.133897,27.849332


In [ ]:
bc = big_countries.merge(final_df.drop(columns="Country ID"), on="Country", how="outer")

In [ ]:
bc[~(bc["Country"].isin(final_df["Country"]))]

,Country,Agricultural Land( %),Land Area(Km2),Birth Rate,Co2-Emissions,CPI,CPI Change (%),Fertility Rate,Forested Area (%),Gasoline Price,...,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude,Democracy Score,Unemployment Rate,Education Index
3,Andorra,0.400,468.0,7.20,469.0,NaN,NaN,1.27,0.340,1.51,...,NaN,NaN,NaN,NaN,67873.0,42.506285,1.521801,NaN,NaN,NaN
5,Antigua and Barbuda,0.205,443.0,15.33,557.0,113.81,0.012,1.99,0.223,0.99,...,NaN,0.165,0.430,NaN,23800.0,17.060816,-61.796428,NaN,NaN,NaN
48,Dominica,0.333,751.0,12.00,180.0,103.87,0.010,1.90,0.574,NaN,...,NaN,0.221,0.326,NaN,50830.0,15.414999,-61.370976,NaN,NaN,NaN
67,Grenada,0.235,349.0,16.47,268.0,107.43,0.008,2.06,0.500,1.12,...,NaN,0.194,0.478,NaN,40765.0,12.116500,-61.679000,NaN,NaN,NaN
73,Vatican City,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,41.902916,12.453389,NaN,NaN,NaN
98,Liechtenstein,0.322,160.0,9.90,51.0,NaN,NaN,1.44,0.431,1.74,...,NaN,NaN,0.216,NaN,5464.0,47.141039,9.520935,NaN,NaN,NaN
107,Marshall Islands,0.639,181.0,29.03,143.0,NaN,NaN,4.05,0.702,1.44,...,NaN,0.178,0.659,NaN,45514.0,7.131474,171.184478,NaN,NaN,NaN
111,Federated States of Micronesia,0.314,702.0,22.82,143.0,112.10,0.005,3.05,0.919,NaN,...,NaN,0.252,0.605,NaN,25963.0,7.425554,150.550812,NaN,NaN,NaN
113,Monaco,NaN,2.0,5.90,NaN,NaN,NaN,NaN,NaN,2.00,...,NaN,NaN,NaN,NaN,38964.0,43.738418,7.424616,NaN,NaN,NaN
120,Nauru,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.522778,166.931503,NaN,NaN,NaN


In [ ]:
bc[~(bc["Country"].isin(big_countries["Country"]))]

,Country,Agricultural Land( %),Land Area(Km2),Birth Rate,Co2-Emissions,CPI,CPI Change (%),Fertility Rate,Forested Area (%),Gasoline Price,...,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude,Democracy Score,Unemployment Rate,Education Index
195,Channel Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.37,NaN
196,French Polynesia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.39,NaN
197,Guam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.98,NaN
198,Hong Kong,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.28,5.32,0.855
199,Macao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.01,NaN
200,New Caledonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.57,NaN
201,Puerto Rico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.27,NaN
202,Taiwan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.99,NaN,NaN
203,Virgin Islands (U.S.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.27,NaN


In [ ]:
bc = bc.drop(columns=[bc.columns[24]])

In [ ]:
bc

,Country,Agricultural Land( %),Land Area(Km2),Birth Rate,Co2-Emissions,CPI,CPI Change (%),Fertility Rate,Forested Area (%),Gasoline Price,...,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Urban_population,Latitude,Longitude,Democracy Score,Unemployment Rate,Education Index
0,Afghanistan,0.581,652230.0,32.49,8672.0,149.90,0.023,4.47,0.021,0.70,...,38041754.0,0.489,0.093,0.714,9797273.0,33.939110,67.709953,0.32,13.28,0.414
1,Albania,0.431,28748.0,11.78,4536.0,119.05,0.014,1.62,0.281,1.36,...,2854191.0,0.557,0.186,0.366,1747593.0,41.153332,20.168331,6.41,11.82,0.745
2,Algeria,0.174,2381741.0,24.28,150006.0,151.36,0.020,3.02,0.008,0.28,...,43053054.0,0.412,0.372,0.661,31510100.0,28.033886,1.659626,3.66,12.70,0.664
3,Andorra,0.400,468.0,7.20,469.0,NaN,NaN,1.27,0.340,1.51,...,77142.0,NaN,NaN,NaN,67873.0,42.506285,1.521801,NaN,NaN,NaN
4,Angola,0.475,1246700.0,40.73,34693.0,261.73,0.171,5.52,0.463,0.97,...,31825295.0,0.775,0.092,0.491,21061025.0,-11.202692,17.873887,3.96,8.53,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Macao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.01,NaN
200,New Caledonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.57,NaN
201,Puerto Rico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.27,NaN
202,Taiwan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.99,NaN,NaN


In [ ]:
bc.to_csv("../full_country_table_cleaned.csv")